Este código es un ejemplo perfecto de un flujo de trabajo completo de NLP (Procesamiento de Lenguaje Natural) "en la vida real". A diferencia de los ejemplos anteriores donde inventábamos frases, aquí descargamos un libro real de internet, lo limpiamos (que es la parte más difícil y sucia) y creamos nuestro motor de traducción a números.

Aquí tienes la explicación estructurada para tu material de apoyo.

# **Taller Avanzado: Procesamiento de Datos Reales (Web Scraping & Cleaning)**

**Objetivo:** Aprender a descargar texto crudo de internet, limpiar la "basura" informática (símbolos raros, errores de codificación) y prepararlo para una Inteligencia Artificial.##

Paso 1: Obtención de Datos (La Fuente)
En lugar de escribir frases manuales, usamos la librería requests para ir a la web del Proyecto Gutenberg y descargar el libro "La Máquina del Tiempo" (The Time Machine).

In [2]:
import numpy as np
import requests
import re
import string

# 1. Descargar texto crudo de internet
# Hacemos una petición (GET) a la URL del archivo de texto
libro = requests.get('https://www.gutenberg.org/files/35/35-0.txt')

# Extraemos solo el contenido de texto
texto = libro.text

print(f"Tipo de dato: {type(texto)}")
print(f"Longitud del texto: {len(texto)} caracteres")

# Vemos un pedazo del texto sucio
print(texto[:2000])

Tipo de dato: <class 'str'>
Longitud del texto: 182973 caracteres
*** START OF THE PROJECT GUTENBERG EBOOK 35 ***




The Time Machine

An Invention

by H. G. Wells


CONTENTS

 I Introduction
 II The Machine
 III The Time Traveller Returns
 IV Time Travelling
 V In the Golden Age
 VI The Sunset of Mankind
 VII A Sudden Shock
 VIII Explanation
 IX The Morlocks
 X When Night Came
 XI The Palace of Green Porcelain
 XII In the Darkness
 XIII The Trap of the White Sphinx
 XIV The Further Vision
 XV The Time Traveller’s Return
 XVI After the Story
 Epilogue




 I.
 Introduction


The Time Traveller (for so it will be convenient to speak of him) was
expounding a recondite matter to us. His pale grey eyes shone and
twinkled, and his usually pale face was flushed and animated. The fire
burnt brightly, and the soft radiance of the incandescent lights in the
lilies of silver caught the bubbles that flashed and passed in our
glasses. Our chairs, being his patents, embraced and caressed us rather

**Nota:** Al imprimir esto, verás mucha "basura" como cabeceras, licencias y quizás caracteres extraños.

## Paso 2: Limpieza Profunda (Data Cleaning)
El texto de internet suele venir con errores de codificación (esos símbolos raros como â\x80\x9c). Esto pasa cuando se mezclan formatos de texto (UTF-8 vs Windows-1252). Si no limpias esto, tu IA aprenderá basura.

In [3]:
# 2. Definir la basura a eliminar
# Estos son códigos de caracteres rotos comunes en textos antiguos de la web
cadenas_a_reemplazar = [
                 '\r\n\r\nâ\x80\x9c', # Nuevo párrafo raro
                 'â\x80\x9c',         # Comilla de apertura rota
                 'â\x80\x9d',         # Comilla de cierre rota
                 '\r\n',              # Salto de línea de Windows
                 'â\x80\x94',         # Guion largo roto
                 'â\x80\x99',         # Apóstrofe roto
                 'â\x80\x98',         # Comilla simple rota
                 '_',                 # Subguiones usados para énfasis
                 ]

# 3. Reemplazar esa basura por espacios
for cadena in cadenas_a_reemplazar:
  # Compilamos la expresión regular para buscar esa cadena exacta
  regexp = re.compile(r'%s' % cadena)
  # Sustituimos por un espacio
  texto = regexp.sub(' ', texto)

# 4. Limpieza final agresiva
# Eliminar cualquier cosa que NO sea código ASCII estándar (quita tildes y ñ si las hubiera)
texto = re.sub(r'[^\x00-\x7F]+', ' ', texto)

# Eliminar números (para que '1999' no sea una palabra)
texto = re.sub(r'\d+', '', texto)

# Convertir todo a minúsculas
texto = texto.lower()

# Veamos cómo quedó ahora más limpio
print(texto[:2000])

*** start of the project gutenberg ebook  ***     the time machine  an invention  by h. g. wells   contents   i introduction  ii the machine  iii the time traveller returns  iv time travelling  v in the golden age  vi the sunset of mankind  vii a sudden shock  viii explanation  ix the morlocks  x when night came  xi the palace of green porcelain  xii in the darkness  xiii the trap of the white sphinx  xiv the further vision  xv the time traveller s return  xvi after the story  epilogue      i.  introduction   the time traveller (for so it will be convenient to speak of him) was expounding a recondite matter to us. his pale grey eyes shone and twinkled, and his usually pale face was flushed and animated. the fire burnt brightly, and the soft radiance of the incandescent lights in the lilies of silver caught the bubbles that flashed and passed in our glasses. our chairs, being his patents, embraced and caressed us rather than submitted to be sat upon, and there was that luxurious after-d

**Concepto Clave:** Aquí es donde limpiarías los Emojis si no los quisieras, o los "Jajajajaja" infinitos.

## **Paso 3:** Parsing (Tokenización Inteligente)
Ahora cortamos el texto en palabras. Usamos la puntuación (puntos, comas, exclamaciones) como tijeras.

In [4]:
# 5. Separar por puntuación
print(f"Signos de puntuación que usaremos para cortar: {string.punctuation}")

# Creamos una regla que diga: "Corta donde haya puntuación O espacios"
puntos_regex = fr'[{string.punctuation}\s]+'

palabras = re.split(puntos_regex, texto)

# 6. Filtrado final
# Quitamos espacios vacíos que hayan quedado
palabras = [item.strip() for item in palabras if item.strip()]

# Quitamos palabras de una sola letra (como 'a', 'y', 'o' en español, o 'I', 'a' en inglés)
# Esto reduce el ruido, aunque a veces se pierde información útil.
palabras = [item for item in palabras if len(item) > 1]

print(f"\nPrimeras 50 palabras procesadas:\n{palabras[:50]}")

Signos de puntuación que usaremos para cortar: !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~

Primeras 50 palabras procesadas:
['start', 'of', 'the', 'project', 'gutenberg', 'ebook', 'the', 'time', 'machine', 'an', 'invention', 'by', 'wells', 'contents', 'introduction', 'ii', 'the', 'machine', 'iii', 'the', 'time', 'traveller', 'returns', 'iv', 'time', 'travelling', 'in', 'the', 'golden', 'age', 'vi', 'the', 'sunset', 'of', 'mankind', 'vii', 'sudden', 'shock', 'viii', 'explanation', 'ix', 'the', 'morlocks', 'when', 'night', 'came', 'xi', 'the', 'palace', 'of']


## **Paso 4:** Estadísticas del Vocabulario
Aquí medimos la riqueza del texto.

In [5]:
# Crear el vocabulario (palabras únicas ordenadas)
vocabulario = sorted(set(palabras))

nPalabras = len(palabras)      # Total de palabras en el libro
nLexico = len(vocabulario)     # Total de palabras ÚNICAS que sabe el sistema

print(f'Total de palabras en el texto: {nPalabras}')
print(f'Tamaño del vocabulario (Tokens únicos): {nLexico}')

Total de palabras en el texto: 30698
Tamaño del vocabulario (Tokens únicos): 4589


## **Paso 5:** Creación del Motor (Encoder/Decoder)
Creamos los diccionarios y las funciones para traducir. Nota que aquí el encoder usa un bucle for, que es más fácil de leer que una list comprehension, y usa numpy para ser más eficiente con grandes datos.

In [6]:
# Diccionarios de mapeo
word2idx = {w:i for i,w in enumerate(vocabulario)}
idx2word = {i:w for i,w in enumerate(vocabulario)}

# Imprimir una muestra del diccionario (cada 87 palabras para ver variedad)
print("Muestra del diccionario:")
for i in list(word2idx.items())[0:10000:87]:
  print(i)

# --- FUNCIÓN ENCODER (Palabras -> Vector Numérico) ---
def encoder(lista_palabras, diccionario_cod):
  # Pre-creamos un vector de ceros del tamaño exacto (más rápido en memoria)
  idxs = np.zeros(len(lista_palabras), dtype=int)

  # Llenamos el vector
  for i, palabra in enumerate(lista_palabras):
    # OJO: Aquí asumimos que la palabra EXISTE en el diccionario.
    # En un sistema real, necesitarías un manejo de errores para palabras desconocidas.
    idxs[i] = diccionario_cod[palabra]

  return idxs

# --- FUNCIÓN DECODER (Vector Numérico -> Texto) ---
def decoder(indices, diccionario_dec):
  return ' '.join([diccionario_dec[i] for i in indices])

Muestra del diccionario:
('abandon', 0)
('aimlessly', 87)
('apologise', 174)
('attained', 261)
('behaved', 348)
('both', 435)
('can', 522)
('cheerfully', 609)
('coat', 696)
('contents', 783)
('culminating', 870)
('delay', 957)
('dimness', 1044)
('dragging', 1131)
('edition', 1218)
('everywhere', 1305)
('facilities', 1392)
('find', 1479)
('footfall', 1566)
('furnishing', 1653)
('gold', 1740)
('hallo', 1827)
('high', 1914)
('ideas', 2001)
('inextinguishable', 2088)
('invest', 2175)
('lamp', 2262)
('likewise', 2349)
('manhood', 2436)
('minerals', 2523)
('mysteries', 2610)
('novelty', 2697)
('outbreaks', 2784)
('paws', 2871)
('plato', 2958)
('previously', 3045)
('questionings', 3132)
('reflecting', 3219)
('return', 3306)
('sandals', 3393)
('senses', 3480)
('shrinking', 3567)
('slit', 3654)
('special', 3741)
('stick', 3828)
('sudden', 3915)
('tap', 4002)
('thrice', 4089)
('treat', 4176)
('unfrozen', 4263)
('vertical', 4350)
('wearisome', 4437)
('wonderful', 4524)


## **Paso 6: Validación y Pruebas**
Probamos que nuestro sistema no rompa el texto. Tomamos un pedazo aleatorio del libro, lo convertimos a números y lo traemos de vuelta.

In [7]:
# 1. Prueba manual simple
print("\n--- Prueba Simple ---")
codificado = encoder(['the', 'time', 'machine'], word2idx)
print(f"Codificado: {codificado}")
print(f"Decodificado: {decoder([1, 3, 10], idx2word)}") # Nota: [1,3,10] son indices inventados para probar


# 2. Prueba de "Ida y Vuelta" (Round-trip)
print("\n--- Prueba de Ida y Vuelta Aleatoria ---")

# Elegimos un punto de partida al azar en el libro
inicio_idx = np.random.choice(nPalabras)

# Tomamos 10 palabras seguidas desde ese punto
indices_secuenciales = np.arange(inicio_idx, inicio_idx + 10)
secuencia_palabras = [ palabras[i] for i in indices_secuenciales ]

print('Texto Original:')
print(secuencia_palabras)

print('\nTokens (Versión numérica):')
secuencia_tokens = encoder(secuencia_palabras, word2idx)
print(secuencia_tokens)

print('\nTexto Decodificado (Reconstrucción):')
print(decoder(secuencia_tokens, idx2word))


--- Prueba Simple ---
Codificado: [4042 4109 2416]
Decodificado: abandoned abnormally absent

--- Prueba de Ida y Vuelta Aleatoria ---
Texto Original:
['again', 'put', 'one', 'more', 'drop', 'of', 'oil', 'on', 'the', 'quartz']

Tokens (Versión numérica):
[  74 3116 2742 2569 1158 2731 2736 2740 4042 3129]

Texto Decodificado (Reconstrucción):
again put one more drop of oil on the quartz
